In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [21]:
import sys
sys.path.append('/home/ext_navish_iitkgp_gmail_com/ganRegression/')

In [4]:
# import numpy as np
# import os
# from functools import partial
# import seaborn as sns
# import plotly.graph_objects as go
# import pandas as pd
# import dask.dataframe as dd
# import matplotlib.pyplot as plt
# import sys
# import pickle
# from pathlib import Path


# # hex2vec
# HOME = os.environ["HOME"]

# sys.path.insert(0, f"{HOME}/hex2vec")

# # add codebase
# sys.path.insert(0, f"/gcsmount-notebook/codebase")

# from src.data.make_dataset import h3_to_polygon
# import urban_tools.constants as uc
# from urban_tools.hex_pipeline import RouteHexHandler, TestTrainManager
# from urban_tools.pipelines import route_hex_pipeline


# ## Read in the Delivery DataFrame
# p = Path("/gcsmount-research-data-staging/osmnx-cities/hexed-routes/debug-multi-synthetic-tags/hh.pkl")
# hh = RouteHexHandler.from_pickle(p)
# # embedding_df = pd.read_pickle("/gcsmount-research-data-staging/osmnx-cities/hexed-complete/Boston, MA/boston_embedding.pkl")

# ### Remove Super Tags
# # hh.drop_super_tags()
# ### Remove Sub Tags
# # hh = hh.drop_sub_tags()
# ### Filter for only H3 with > X Data Points

# hh = hh.filter_hex_occurance(20)
# ## Drop Unecessary Columns
# hh.other_tags
# ### Drop Chicago
# # print(tagged_df.shape[0].compute(), tagged_df.shape[1])
# # tagged_df = tagged_df.loc[~tagged_df["city"].str.contains("Chicago")]
# # filter for only Boston
# hh.df = hh.df.loc[hh.df.city.str.contains("Boston")]
# # print(tagged_df.shape[0].compute(), tagged_df.shape[1])

# ## Data Preprocessing: Scaling and Splitting into train test


# # create a grouped dataframe
# tagged_df = hh.df.groupby("h3").agg({
#     "planned_service_time_log": "mean",
#     **{
#         tag: "first"
#         for tag in hh.all_tags
#     }
# })


# from sklearn.preprocessing import RobustScaler

# tt = TestTrainManager(
#     tagged_df,
#     x_col=hh.all_tags,
#     y_col=["planned_service_time_log"],
#     scaler=RobustScaler,
#     grouped=True,
# )
# tt.split_test_train(train_size=0.8, random_seed=6781)


# tt.scale_test_train()
# tt.build_test_df(agg=False)

In [5]:
import os
import sys
import pickle
from pathlib import Path
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from functools import partial
import seaborn as sns
import gc
from numpy.random import binomial
from numpy.random import normal

import pandas as pd
import seaborn as sns

# %reset -s -f
import os
from functools import partial
import logging

# hex2vec
HOME = os.environ["HOME"]

sys.path.insert(0, f"{HOME}/hex2vec")

# add codebase
sys.path.insert(0, f"/gcsmount-notebook/codebase")

from src.data.make_dataset import h3_to_polygon
import urban_tools.constants as uc
from urban_tools.hex_pipeline import RouteHexHandler, TestTrainManager
from urban_tools.pipelines import route_hex_pipeline

p = Path("/gcsmount-research-data-staging/osmnx-cities/hexed-routes/debug-multi-synthetic-tags/hh.pkl")
hh = RouteHexHandler.from_pickle(p)
# embedding_df = pd.read_pickle("/gcsmount-research-data-staging/osmnx-cities/hexed-complete/Boston, MA/boston_embedding.pkl")

### Filter for only H3 with > X Data Points
hh = hh.filter_hex_occurance(20)
## Drop Unecessary Columns
hh.other_tags

## Data Preprocessing: Scaling and Splitting into train test

#### Create the Test-Train Manager
# filter for only Boston
from sklearn.preprocessing import RobustScaler
hh.df = hh.df.loc[hh.df.city.str.contains("Boston")]
tt = TestTrainManager(
        hh.df,
        scaler=RobustScaler,
        x_col=hh.all_tags,
        y_col=hh.df.columns.intersection(["planned_service_time_log"]),
    )

tt.split_test_train(random_seed=12323)
tt.scale_test_train()
tt.build_test_df()
train = pd.concat([tt.X_train, tt.Y_train], axis=1, copy=True)
test = pd.concat([tt.X_test, tt.Y_test.loc, tt.Y_test.scale], axis=1, copy=True)

print(f'Training Data shape is {tt.X_train.shape}')
print(f'testing Data shape is {tt.X_test.shape}')

del hh
gc.collect()

# convert a pandas dataframe to a pytorch dataset
class PandasDataset(torch.utils.data.Dataset):
    def __init__(self, x_df, y_df):
        self._x_df = x_df.values.astype(np.float32)
        self._y_df = y_df.values.astype(np.float32).ravel()

    def __len__(self):
        return len(self._x_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self._x_df[idx], self._y_df[idx]

Training Data shape is (28337, 674)
testing Data shape is (240, 674)


In [23]:
# !pip uninstall umapy
# !pip install umap-learn
# !pip install hdbscan

In [53]:
import numpy as np
import pandas as pd

from mdn_model import MDN

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

In [11]:
X_train, y_train, X_test, y_test = torch.Tensor(tt.X_train.values), torch.Tensor(tt.Y_train.values.ravel()), torch.Tensor(tt.X_test.values), torch.Tensor(tt.Y_test.loc.values)

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([28337, 674]),
 torch.Size([28337]),
 torch.Size([240, 674]),
 torch.Size([240]))

## Stochastic Variational GP Regression

In [13]:
import tqdm
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

In [14]:
import urllib.request
import os
from scipy.io import loadmat
from math import floor

In [15]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

In [17]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

inducing_points = X_train[:500, :]
model = GPModel(inducing_points=inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()

In [21]:
# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)

num_epochs = 1 if smoke_test else 4


model.train()
likelihood.train()

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=1e-4)

# Our loss object. We're using the VariationalELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=y_train.size(0))


epochs_iter = tqdm.notebook.tqdm(range(num_epochs), desc="Epoch")
for i in epochs_iter:
    # Within each iteration, we will go over each minibatch of data
    minibatch_iter = tqdm.notebook.tqdm(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
        optimizer.zero_grad()
        output = model(x_batch)
        loss = -mll(output, y_batch)
        minibatch_iter.set_postfix(loss=loss.item())
        loss.backward()
        optimizer.step()

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/28 [00:00<?, ?it/s]

NotPSDError: Matrix not positive definite after repeatedly adding jitter up to 1.0e-06.